In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

In [3]:
from src.data import read_raw_data


data = read_raw_data()

In [4]:
data['weather'][data['weather']['AvgSpeed']=='M']

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
87,2,2007-06-13,86,68,77,M,53,62,0,12,...,,M,M,M,0.00,M,M,7.0,5,M
1745,2,2011-09-14,60,48,54,M,45,51,11,0,...,RA BR HZ FU,M,M,M,T,29.47,M,6.0,32,M
2067,2,2012-08-22,84,72,M,M,51,61,M,M,...,,M,M,M,0.00,29.39,M,4.7,19,M


In [5]:
# predict AvgSpeeed using ResultSpeed and ResultDir

from sklearn.linear_model import LinearRegression

X_train = data['weather'][data['weather']['AvgSpeed']!='M'][['ResultSpeed', 'ResultDir', 'AvgSpeed']].astype(float)
y_train = X_train.pop('AvgSpeed')

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

X_test = data['weather'][data['weather']['AvgSpeed']=='M'][['ResultSpeed', 'ResultDir']]
missing_value_predictions = lr_model.predict(X_test).round(1)

m_index = data['weather'][data['weather']['AvgSpeed']=='M'].index
data['weather'].loc[m_index, 'AvgSpeed'] = missing_value_predictions

data['weather']['AvgSpeed'] = data['weather']['AvgSpeed'].astype(np.float64)

In [6]:
from src.data import clean_weather


data['weather'] = clean_weather(data['weather'])

In [7]:
from src.data import build_data_preprocessing_pipeline


# build preprocessing pipeline
data_preprocessing_pipeline = build_data_preprocessing_pipeline()

# preprocess train and test data
data['train'] = data_preprocessing_pipeline.fit_transform(data['train'])
data['test'] = data_preprocessing_pipeline.transform(data['test'])

[Pipeline]  (step 1 of 3) Processing functiontransformer-1, total=   0.0s
[Pipeline]  (step 2 of 3) Processing monthspeciestraptransformer, total=   0.0s
[Pipeline]  (step 3 of 3) Processing functiontransformer-2, total=   0.0s


In [8]:
from src.features import SpeciesEncoder


species_oh_encoder = SpeciesEncoder()
data['train'] = species_oh_encoder.fit_transform(data['train'])
data['test'] = species_oh_encoder.transform(data['test'])

In [9]:
data['train'] = data['train'][['Date', 'Latitude', 'Longitude', 'Dayofyear',
       'Species_CULEX PIPIENS', 'Species_CULEX PIPIENS/RESTUANS',
       'Species_CULEX RESTUANS', 'WnvPresent']]
data['test'] = data['test'][['Date', 'Id', 'Latitude', 'Longitude', 'Dayofyear',
       'Species_CULEX PIPIENS', 'Species_CULEX PIPIENS/RESTUANS',
       'Species_CULEX RESTUANS']]

In [10]:
data['train'] = data['train'].merge(data['weather'], on='Date')
data['test'] = data['test'].merge(data['weather'], on='Date')
data['train'].drop('Date', axis=1, inplace=True)
data['test'].drop('Date', axis=1, inplace=True)

In [11]:
X_train = data['train'].copy()
y_train = X_train.pop('WnvPresent')

print(y_train.mean().round(4))

0.0545


In [12]:
# from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import TomekLinks, EditedNearestNeighbours, RandomUnderSampler, NeighbourhoodCleaningRule
from imblearn.over_sampling import SMOTE, ADASYN

from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, RepeatedStratifiedKFold

from src.evaluate import evaluate


classifiers = {
    'lr': LogisticRegression(),
    # 'svc': SVC(probability=True),
    # 'rand_forest': RandomForestClassifier(),
    'lgbm_unb': LGBMClassifier(is_unbalance=True),
    'lgbm': LGBMClassifier(is_unbalance=False),
    # 'xgb': XGBClassifier(),
    # 'bag_lr': BaggingClassifier(LogisticRegression(), n_estimators=100, max_samples=0.8, max_features=0.5)
}

def run_one_clf(clf):

    steps = [
        ('scaler', StandardScaler()),
        # ('smote', SMOTE(sampling_strategy=0.2)),
        ('adasyn', ADASYN(sampling_strategy=0.3)),
        # ('tomek', TomekLinks()),
        ('enn', EditedNearestNeighbours()),
        ('clf', clf)
    ]

    pipeline = Pipeline(steps=steps)
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)

    cv_results = cross_validate(pipeline, X_train, y_train, scoring='roc_auc', cv=cv)

    val_score = cv_results['test_score'].mean().round(4)
    
    pipeline.fit(X_train, y_train)
    public_score, private_score = evaluate(pipeline, data['test'], X_train.columns.to_list())
    
    return private_score, public_score, str(val_score)

In [14]:
import pandas as pd


results_df = pd.DataFrame(columns=['private_score', 'public_score', 'val_score'])

for name, clf in classifiers.items():
    results_df.loc[name] = run_one_clf(clf)

/home/aga/repos/west_nile_virus/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


100%|██████████| 1.79M/1.79M [00:01<00:00, 1.16MB/s]
2024-09-28 13:51:25.119 | DEBUG    | src.evaluate:evaluate:56 - File pushed to Kaggle API


400 - Bad Request - Submission not allowed:  Your team has used its daily Submission allowance (5) today, please try again tomorrow UTC (11 hours from now).


2024-09-28 13:51:27.018 | DEBUG    | src.evaluate:evaluate:73 - Kaggle scores logged to mlfmlow.


[LightGBM] [Info] Number of positive: 2269, number of negative: 6454
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000916 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4009
[LightGBM] [Info] Number of data points in the train set: 8723, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.260117 -> initscore=-1.045361
[LightGBM] [Info] Start training from score -1.045361
[LightGBM] [Info] Number of positive: 2140, number of negative: 6419
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000755 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4165
[LightGBM] [Info] Number of data points in the train set: 8559, number of used features: 20
[LightGBM] [Info] [binar

...

2024-09-28 13:51:49.232 | DEBUG    | src.evaluate:evaluate:73 - Kaggle scores logged to mlfmlow.


[LightGBM] [Info] Number of positive: 2166, number of negative: 6417
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001184 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4003
[LightGBM] [Info] Number of data points in the train set: 8583, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.252359 -> initscore=-1.086069
[LightGBM] [Info] Start training from score -1.086069
[LightGBM] [Info] Number of positive: 2144, number of negative: 6453
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000918 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3998
[LightGBM] [Info] Number of data points in the train set: 8597, number of used features: 20
[LightGBM] [Info] [binar

100%|██████████| 1.81M/1.81M [00:01<00:00, 1.22MB/s]
2024-09-28 13:52:06.691 | DEBUG    | src.evaluate:evaluate:56 - File pushed to Kaggle API


400 - Bad Request - Submission not allowed:  Your team has used its daily Submission allowance (5) today, please try again tomorrow UTC (11 hours from now).


2024-09-28 13:52:08.748 | DEBUG    | src.evaluate:evaluate:73 - Kaggle scores logged to mlfmlow.


In [14]:
results_df

,private_score,public_score,val_score
lr,0.69362,0.70306,0.7636
lgbm_unb,0.69362,0.70306,0.8234
lgbm,0.69362,0.70306,0.8209


In [202]:
results_df

,private_score,public_score,val_score
lr,0.63004,0.64238,0.7658
lgbm_unb,0.71196,0.72452,0.8219
lgbm,0.71501,0.72811,0.8191


In [199]:
results_df

,private_score,public_score,val_score
lr,0.64199,0.65169,0.7669
lgbm_unb,0.72579,0.73338,0.8247
lgbm,0.70161,0.70123,0.8213


In [190]:
results_df

,private_score,public_score,val_score
lr,0.63936,0.64902,0.7662
lgbm_unb,0.73591,0.74634,0.8237
lgbm,0.68611,0.70312,0.8248


In [186]:
results_df

,private_score,public_score,val_score
lr,0.65514,0.66740,0.7633
lgbm_unb,0.71874,0.73564,0.8276
lgbm,0.70479,0.72474,0.8296


In [152]:
results_df

,private_score,public_score,val_score
lr,0.65697,0.66909,0.7623
bag_lr,0.67855,0.68860,0.7461


In [149]:
results_df

,private_score,public_score,val_score
lr,0.65697,0.66909,0.7629
bag_lr,0.68191,0.69084,0.7457


In [145]:
results_df

,private_score,public_score,val_score
lr,0.65697,0.66909,0.7627
rand_forest,0.66412,0.69276,0.7692
lgbm,0.72206,0.73312,0.833
xgb,0.70955,0.73028,0.8275
bag_lr,0.67325,0.68345,0.7526


In [142]:
results_df

,private_score,public_score,val_score
lr,0.65697,0.66909,0.7644
rand_forest,0.67081,0.69972,0.7678
lgbm,0.72206,0.73312,0.8326
xgb,0.70955,0.73028,0.8236
bag_lr,0.65703,0.67004,0.7575


In [139]:
# feature_fraction=0.6
results_df

,private_score,public_score,val_score
lr,0.65697,0.66909,0.7637
rand_forest,0.67031,0.70529,0.764
lgbm,0.72206,0.73312,0.8331
xgb,0.70955,0.73028,0.8274


In [136]:
# is_unbalanced in lgbm
results_df

,private_score,public_score,val_score
lr,0.65697,0.66909,0.7621
rand_forest,0.67211,0.71738,0.7726
lgbm,0.70962,0.72971,0.832
xgb,0.70955,0.73028,0.8291


In [132]:
# no scaling
results_df

,private_score,public_score,val_score
lr,0.60649,0.60403,0.742
rand_forest,0.64914,0.68921,0.7675
lgbm,0.71065,0.73095,0.8312
xgb,0.70955,0.73028,0.8297


In [129]:
# with scaling
results_df

,private_score,public_score,val_score
lr,0.65697,0.66909,0.7634
rand_forest,0.64848,0.68473,0.7687
lgbm,0.70962,0.72971,0.8308
xgb,0.70955,0.73028,0.8268


In [166]:
lgbm = LGBMClassifier(is_unbalance=True)
lgbm.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'is_unbalance': True}